# 示例：从个人广告中获取区域倾向

In [1]:
import feedparser
ny = feedparser.parse('https://newyork.craigslist.org/d/rooms-shares/search/roo?format=rss')
print (len(ny['entries']))

25


In [2]:
ny = feedparser.parse('https://sfbay.craigslist.org/d/rooms-shares/search/roo?format=rss')
print (len(ny['entries']))

25


In [3]:
ny = feedparser.parse('https://newyork.craigslist.org/d/rooms-shares/search/roo?format=rss')
print (len(ny['entries']))
for entry in ny['entries']:
    print (entry['summary_detail']['value'])

25
* Bedroom for rent in a 3BR/1Bath apt, 866/mo + utilities (utilities/internet not included), Central Park North and museum mile, fully furnished with desk and loft bed/full mattress. A five-minute walk to the 6 train and a ten-minute walk to the 2/3  ...
Sunny Room with good sized closet. 
Sublet for June 1st, move in date flexible (could be end of May). No Fees! 
This is a sublet, June 1- May 31, 2021 with a year sublease! 
Ideal sublet for one full year! 
Please email dates you are interested in an ...
*** or ***
*** or ***
Single furnished room with all utilities included for $850 per month. Available immediately. Bathroom and kitchen to be shared with 2 other male housemates. 1 month security+ 1 month advance rent to move in. W2, paystubs, Work and old landlord refere ...
*** or ***
*** or ***
Bedroom available in brick house built in 1990. Shared access to living room, dining room, and recently renovated gourmet kitchen with large island, with patio door access to back porch. O

# 收集数据

In [4]:
import random
from numpy import *
import feedparser
import re
import operator  
import numpy as np

In [5]:
def calcMostFreq(vocabList, fullText):
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token] = fullText.count(token)
    sortedFreq = sorted(freqDict.items(), key=operator.itemgetter(1), reverse=True)
    return sortedFreq[:30] #返回排序最高的30词

In [6]:
def localWords(feed1,feed0):
    import feedparser
    docList=[]
    classList = []
    fullText = []
    minLen = min(len(feed1['entries']), len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)  # NY is class 1
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)   # create vocabulary
    
    top10Words = calcMostFreq(vocabList, fullText)   # remove top 30 words
    for pairW in top10Words:
        if pairW[0] in vocabList:
            vocabList.remove(pairW[0])
            
    
    trainingSet = list(range(2 * minLen)); testSet = []
    for i in range(5):
        randIndex = int(np.random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])    
        del(trainingSet[randIndex])
        
   
    trainMat = []; trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))  #将训练集中的每一条数据，转化为词向量
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))  #开始训练

    
    errorCount = 0
    for docIndex in testSet:        # classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is: ', float(errorCount)/len(testSet))
    return vocabList, p0V, p1V

# 准备数据

In [7]:
def textParse(bigString):
    import re
    # 使用正则表达式来切分句子，去掉少于两个字符的字符串，并变成小写
    listOfTokens =re.split(r'\W*',bigString)
    return [tok.lower() for tok in listOfTokens if len(tok)>2]

In [8]:
def createVocabList(dataSet):
    vocabSet = set([])        
    for document in dataSet:
        vocabSet = vocabSet | set(document) 
    return list(vocabSet)

In [13]:
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)    
    for word in inputSet:            
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

In [15]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    #p0Num = zeros(numWords); p1Num = zeros(numWords)     #初始化概率  
    #p0Denom = 0.0; p1Denom = 0.0 
    p0Num = ones(numWords); p1Num = ones(numWords)      
    p0Denom = 2.0; p1Denom = 2.0                       
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    #p1Vect = p1Num/p1Denom         #change to log()
    #p0Vect = p0Num/p0Denom         #change to log()
    p1Vect = log(p1Num/p1Denom)          
    p0Vect = log(p0Num/p0Denom)   
    return p0Vect,p1Vect,pAbusive
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)   #2分类问题可以用1.0-pClass1，多分类需要修改
    if p1 > p0:
        return 1
    else: 
        return 0

In [10]:
ny=feedparser.parse('https://newyork.craigslist.org/d/rooms-shares/search/roo?format=rss')

In [11]:
sf=feedparser.parse('https://newyork.craigslist.org/d/rooms-shares/search/roo?format=rss')

In [16]:
vocabList, pSF, pNY =localWords(ny, sf)

the error rate is:  0.6


C:\Users\lenovo\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [17]:
vocabList, pSF, pNY =localWords(ny, sf)

the error rate is:  0.8


C:\Users\lenovo\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [22]:
def getTopWords(ny,sf):
    import operator
    vocabList, p0V, p1V=localWords(ny,sf)
    topNY=[];topSF=[]
    for i in range(len(p0V)):
        if p0V[i] > -6.0:
            topSF.append((vocabList[i],p0V[i]))
        if p1V[i] > -6.0:
            topNY.append((vocabList[i],p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair: pair[1], reverse=True)
    print("SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**")
    for item in sortedSF:
        print(item[0])
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print("NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**")
    for item in sortedNY:
        print(item[0])

In [23]:
getTopWords(ny,sf)

the error rate is:  1.0
SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**
nice
una
house
mudese
guarantor
big
serian
walk
sized
income
june
windows
apt
llamame
closets
credit
not
650
rooms
private
este
limpia
trabajando
new
hoy
minute
rent
mismo
719
bedrooms
brand
good
fully
responsable
actualmente
access
year
para
775
internet
entrar
security
sea
desk
space
patio
back
bathroom
second
ten
share
west
recently
all
cabinet
2bedroom
district
other
skip
ideal
advance
deposit
plan
lorimer
environment
door
bed
work
floors
mattress
1bath
mate
off
flexible
porch
sun
pay
cable
professional
per
broadway
first
3br
750
female
jmz
renovated
train
duplex
1st
built
sunlight
modern
30k
five
spacious
museum
hardwood
could
are
every
fees
700
section
city
persona
sublease
pareja
housemates
mile
immediately
you
calm
single
lots
neighborhood
this
one
balcony
long
850
high
speed
term
tons
end
bright
2021
email
1990
park
925
closet
just
central
date
stove
quiet
jersey
island
866
dates
paystubs


C:\Users\lenovo\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
